## 第 4 章 线性回归

线性回归用一条“最合适的直线（或超平面）”去拟合数据，用来**预测数值**或者**分析变量之间的影响关系**。

## 4.1 线性回归简介

### 4.1.1 什么是线性回归

**定义**

线性回归假设因变量 $y$ 和自变量 $x_1, x_2, \dots, x_n$ 存在近似线性关系：

$$
y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \dots + \beta_n x_n
$$

- $\beta_0$：截距，表示当所有自变量都为 0 时，模型给出的“基准值”
- $\beta_1, \dots, \beta_n$：系数，表示每个自变量对 $y$ 的影响大小和方向（正相关 / 负相关）

模型的目标：**选出一组最合适的系数 $\beta$**，让“预测值”尽量接近“真实值”。

**一元线性回归**

只有一个自变量 (x) 时：

$$
y = \beta_0 + \beta_1 x
$$

就是在平面直角坐标系中，**找一条最合适的直线**。

**多元线性回归**

有多个自变量时：

$$
y = \beta_0 + \beta_1 x_1 + \dots + \beta_n x_n
$$

这时几何上对应的是一个**超平面**，但理解上和一元情况是一样的：用线性的形式去拟合数据。

**应用场景（直观理解）**

* GDP 预测：用投资、消费、出口等指标预测 GDP 增长
* 广告效果评估：分析 TV、Radio、Newspaper 投放对销售额的影响
* 药物剂量研究：剂量变化会让血压、血糖变化多少
* 产品质量控制：工艺参数（温度、压力等）对产品强度、寿命的影响
* 政策效果评估：最低工资、环保政策对就业率、排放量的影响
* 气候变化：排放量、森林覆盖率等对气温的影响

一句话总结：**线性回归用来回答“这个量变一点，那个量会大概变多少？”的问题。**

### 4.1.2 线性回归 API 使用示例（sklearn）

需求：
某中学教师想研究**学生每周学习时间（小时）**和**数学成绩（0–100 分）**之间的关系，并用模型来**预测成绩**。

In [1]:
from sklearn.linear_model import LinearRegression

# 自变量，每周学习时长（单位：小时）
X = [[5], [8], [10], [12], [15], [3], [7], [9], [14], [6]]

# 因变量，数学考试成绩（单位：分）
y = [55, 65, 70, 75, 85, 50, 60, 72, 80, 58]

# 1. 创建线性回归模型
model = LinearRegression()

# 2. 训练模型，让它“学会”从学习时长预测成绩
model.fit(X, y)

# 3. 查看学到的参数
print("系数 beta1:", model.coef_)        # 每多学 1 小时，成绩大约多多少分
print("截距 beta0:", model.intercept_)   # 不学习时（x=0）模型预估的基准分

# 4. 预测：每周学习 11 小时，可能考多少分
print("学习 11 小时的预测成绩:", model.predict([[11]])[0])

系数 beta1: [2.87070855]
截距 beta0: 41.45069393718042
学习 11 小时的预测成绩: 73.02848794740687


**结果说明**

* `coef_`（系数）：大于 0 表示“多学一点，分数涨一点”；数值越大，影响越大
* `intercept_`（截距）：没有学习时间时的“理论成绩”，只是模型拟合出来的一个数，不一定真实存在
* `predict([[11]])`：输入 11 小时，输出预测成绩，比如大约 75–80 分

**应用场景**

当你有一组“输入–输出”的历史数据（学习时间 → 成绩），线性回归可以帮你**学出一个简单的公式**，用于**预测未来**和**解释影响关系**。

## 4.2 线性回归求解

### 4.2.1 损失函数：用来衡量“拟合得好不好”

模型的预测值和真实值一般不会完全相同，需要一个指标来衡量误差，这个指标就是**损失函数**。

设：

* 样本数：$n$
* 第 $i$ 个样本的真实值：$y_i$
* 模型的预测值：$f(x_i)$

#### 1）均方误差（MSE）

> 回归问题中最常用的损失函数。

$$
\mathrm{MSE} = \frac{1}{n} \sum_{i=1}^n \bigl(f(x_i) - y_i\bigr)^2
$$

特点：

* **大误差惩罚更重**：误差会被平方，比如误差从 2 变 10，平方从 4 变 100
* **是凸函数**：只有一个全局最小值，且处处可导，方便用解析解或梯度下降求解
* 在线性回归中，对 MSE 最小化的方法就是**最小二乘法**
* 几何意义：**让所有点到直线（或超平面）的“垂直距离”的平方和最小**

#### 2）MSE 与极大似然估计（MLE）的关系（偏理论）

假设真实数据满足：

$$
y_i = \beta^\top x_i + \varepsilon_i
$$

其中误差项：

* $\varepsilon_i$：**独立同分布**并且服从正态分布

$$
\varepsilon_i \sim \mathcal{N}(0, \sigma^2)
$$

给定 $x_i$ 和参数 $\beta$，$y_i$ 的条件概率为：

$$
p(y_i \mid x_i; \beta) =
\frac{1}{\sqrt{2\pi\sigma^2}}
\exp\left(- \frac{(y_i - \beta^\top x_i)^2}{2\sigma^2}\right)
$$

全部样本的**似然函数**：

$$
L(\beta) = \prod_{i=1}^n p(y_i \mid x_i; \beta)
$$

取对数得到对数似然函数：

$$
\ln L(\beta) =
-\frac{n}{2}\ln(2\pi\sigma^2)
-\frac{1}{2\sigma^2}
\sum_{i=1}^n (y_i - \beta^\top x_i)^2
$$

最大化 $\ln L(\beta)$ 等价于最小化：

$$
\sum_{i=1}^n (y_i - \beta^\top x_i)^2
$$

也就是最小化 MSE。

**结论：**
当误差服从高斯分布时，线性回归使用 MSE 作为损失函数是理论最优的选择。

#### 3）平均绝对误差（MAE）

> 数据中有大量异常值（outliers）时更稳健的选择。

$$
\mathrm{MAE} = \frac{1}{n} \sum_{i=1}^n |f(x_i) - y_i|
$$

特点：

* 对**异常值不敏感**：因为误差是线性惩罚，不会被平方放大
* 对**小误差**的惩罚比 MSE 更弱
* 常用于金融风险、鲁棒回归等对异常值敏感的场景


### 4.2.2 一元线性回归解析解（手算公式）

考虑一元线性回归：

$$
f(x_i) = \beta_0 + \beta_1 x_i
$$

目标：最小化 MSE：

$$
\mathrm{MSE} = \frac{1}{n} \sum_{i=1}^n (\beta_0 + \beta_1 x_i - y_i)^2
$$

对 $\beta_0, \beta_1$ 求偏导、令导数为 0，可以得到闭式解（推导过程略）：

* 记：

  $$
  \bar x = \frac{1}{n} \sum_{i=1}^n x_i,\quad
  \bar y = \frac{1}{n} \sum_{i=1}^n y_i
  $$

* 则系数为：

  $$
  \beta_1 =
  \frac{\sum_{i=1}^n (x_i - \bar x)(y_i - \bar y)}
  {\sum_{i=1}^n (x_i - \bar x)^2}, \quad
  \beta_0 = \bar y - \beta_1 \bar x
  $$

**例子：学习时间与成绩**

用同样的数据：

```python
X = [5, 8, 10, 12, 15, 3, 7, 9, 14, 6]
y = [55, 65, 70, 75, 85, 50, 60, 72, 80, 58]
```

计算得到大致结果（四舍五入）：

* $\beta_1 \approx 2.87$：多学 1 小时，成绩大约多 2.87 分
* $\beta_0 \approx 41.45$：理论上的“0 小时学习”成绩

即模型为：

$$
\hat y \approx 41.45 + 2.87 x
$$

**应用场景**

样本量不大、只有一个特征时，直接用公式就能快速算出最优直线，无需写循环或用优化算法。

### 4.2.3 正规方程法（矩阵形式的解析解）

当特征数较多时，我们通常用矩阵写法。

定义：

* 特征矩阵 $X$（在第一列加上一列全 1 对应截距）：

  $$
  X = \begin{bmatrix}
  1 & x_{11} & x_{12} & \dots & x_{1m} \
  1 & x_{21} & x_{22} & \dots & x_{2m} \
  \vdots & \vdots & \vdots & \ddots & \vdots \
  1 & x_{n1} & x_{n2} & \dots & x_{nm}
  \end{bmatrix}
  $$

* 参数向量 $\beta$：

  $$
  \beta =
  \begin{bmatrix}
  \beta_0 \
  \beta_1 \
  \vdots \
  \beta_m
  \end{bmatrix}
  $$

* 目标向量 $y$：

  $$
  y =
  \begin{bmatrix}
  y_1 \
  y_2 \
  \vdots \
  y_n
  \end{bmatrix}
  $$

预测写成矩阵形式：

$$
\hat y = X \beta
$$

MSE（忽略常数 $\frac{1}{n})$可写成：

$$
J(\beta) = |X\beta - y|_2^2 = (X\beta - y)^\top (X\beta - y)
$$

对 $\beta$ 求导并令导数为 0，可得**正规方程**：

$$
X^\top X \beta = X^\top y
$$

只要 $X^\top X$ 可逆，就有解析解：

$$
\beta = (X^\top X)^{-1} X^\top y
$$

> 正规方程适合：样本数较多但**特征数量不太大**的情况。
> 特征太多时，求逆 $(X^\top X)^{-1}$ 计算量很大，此时更适合用梯度下降等迭代优化方式。

**sklearn 中的 LinearRegression（背后就是用解析解）**

In [2]:
from sklearn.linear_model import LinearRegression

# 三个简单样本
X = [[0, 3],
     [1, 2],
     [2, 1]]
y = [0, 1, 2]

model = LinearRegression(fit_intercept=True)  # 是否包含截距
model.fit(X, y)

print("系数 coef_:", model.coef_)        # 对应每一列特征的系数
print("截距 intercept_:", model.intercept_)

系数 coef_: [ 0.5 -0.5]
截距 intercept_: 1.4999999999999996


**应用场景**

当数据规模中等，且你希望**快速直接得到解析解**时，用 `LinearRegression` 非常方便。

### 4.2.4 梯度下降法（数值优化思路）

#### 1）基本更新公式

梯度下降（Gradient Descent）是一种**迭代优化算法**。
核心思想：**沿着损失函数的负梯度方向，一步一步往下走**。

设目标函数为 $J(\beta)$，参数为 $\beta$，在第 $t$ 次迭代时：

$$
\beta^{(t+1)} = \beta^{(t)} - \alpha \cdot \nabla J\bigl(\beta^{(t)}\bigr)
$$

* $\alpha$：学习率（learning rate），控制每一步走多大
* $\nabla J(\beta)$：在 $\beta$ 处的梯度（对各个参数的偏导）

**在线性回归 + MSE 的场景中**，若使用矩阵形式
$J(\beta) = \dfrac{1}{n}\|X\beta - y\|_2^2$，可求得：

$$
\nabla_\beta J(\beta) = \frac{2}{n} X^\top (X\beta - y)
$$

这就是更新时要用的梯度。

#### 2）计算示例（学习时间与成绩）

仍然使用前面的数据，构造矩阵：

* 原始特征：$x \in \mathbb{R}^{n \times 1}$

* 添加一列全 1 得到：

  $$
  X = \begin{bmatrix}
  1 & x_1 \\
  1 & x_2 \\
  \vdots & \vdots \\
  1 & x_n
  \end{bmatrix}
  $$

* 参数：

  $$
  \beta =
  \begin{bmatrix}
  \beta_0 \\
  \beta_1
  \end{bmatrix}
  $$

* 损失函数：

  $$
  J(\beta) = \frac{1}{n}\|X\beta - y\|_2^2
  $$

* 梯度：

  $$
  \nabla J(\beta) = \frac{2}{n} X^\top (X\beta - y)
  $$

选择：

* 初始参数：$\beta^{(0)} = [1, 1]^\top$
* 学习率：$\alpha = 0.01$

不断按公式更新：

$$
\beta^{(t+1)} = \beta^{(t)} - \alpha \cdot \frac{2}{n} X^\top (X\beta^{(t)} - y)
$$

经过多次迭代后，$\beta$ 会逐渐逼近解析解：
$\beta_0 \approx 41.45,\ \beta_1 \approx 2.87$，损失 $J(\beta)$ 越来越小。

---

**实现代码示例（Numpy）**


In [3]:
import numpy as np

# 原始数据
X = np.array([[5], [8], [10], [12], [15], [3], [7], [9], [14], [6]])   # 学习时长
y = np.array([[55], [65], [70], [75], [85], [50], [60], [72], [80], [58]])  # 成绩

n = X.shape[0]

# 在 X 前面加一列全 1，对应偏置 beta0
X = np.hstack([np.ones((n, 1)), X])

# 初始化参数 beta0, beta1 都为 1
beta = np.array([[1.0], [1.0]])

def J(beta):
    """目标函数：MSE"""
    return np.sum((X @ beta - y) ** 2) / n

def gradient(beta):
    """梯度"""
    return 2 * X.T @ (X @ beta - y) / n

alpha = 1e-2   # 学习率
max_epoch = 10000

for epoch in range(1, max_epoch + 1):
    j = J(beta)
    grad = gradient(beta)
    beta = beta - alpha * grad

    if epoch % 1000 == 0:
        print(f"epoch={epoch}, beta={beta.reshape(-1)}, J={j}")

print("最终 beta:", beta.reshape(-1))

epoch=1000, beta=[39.30858775  3.07624954], J=3.6613222868937436
epoch=2000, beta=[41.33570192  2.88174235], J=2.983114188688514
epoch=3000, beta=[41.44452096  2.87130086], J=2.981159775470073
epoch=4000, beta=[41.45036256  2.87074034], J=2.9811541433771502
epoch=5000, beta=[41.45067615  2.87071025], J=2.981154127146987
epoch=6000, beta=[41.45069298  2.87070864], J=2.981154127100208
epoch=7000, beta=[41.45069389  2.87070855], J=2.981154127100079
epoch=8000, beta=[41.45069393  2.87070855], J=2.981154127100084
epoch=9000, beta=[41.45069394  2.87070855], J=2.981154127100067
epoch=10000, beta=[41.45069394  2.87070855], J=2.981154127100069
最终 beta: [41.45069394  2.87070855]


**学习率的影响**

* 学习率过大：可能“跳过”最优点，甚至发散
* 学习率过小：每次走得太小，收敛很慢
* 实战中常用：

  * 学习率衰减策略
  * 自适应优化器（Adam、Adagrad 等）

**实践建议**

* 在使用梯度下降前，常常需要对特征做**标准化 / 归一化**，可以极大提升收敛速度
* 对复杂损失面，可能会遇到局部最小值或鞍点，可以借助动量、Adam 等优化方法

**sklearn 中用梯度方式训练线性回归：SGDRegressor**

In [4]:
from sklearn.linear_model import SGDRegressor

X = [[0, 3], [1, 2], [2, 1]]
y = [0, 1, 2]

model = SGDRegressor(
    loss="squared_error",   # 损失函数：平方误差
    fit_intercept=True,     # 是否学习截距
    learning_rate="constant",
    eta0=0.1,               # 初始学习率
    max_iter=1000,
    tol=1e-8                # 损失不再明显下降时提前停止
)
model.fit(X, y)

print("系数 coef_:", model.coef_)
print("截距 intercept_:", model.intercept_)

系数 coef_: [ 0.90900056 -0.090926  ]
截距 intercept_: [0.27286632]


**应用场景**

* 特征维度很高、样本巨大时（如推荐系统、CTR 预估），**正规方程求逆几乎不可行**，此时用 SGD/mini-batch SGD 等梯度方法更实际。

## 4.3 案例：广告投放效果预测

### 4.3.1 数据集说明

使用 Kaggle 上的 **Advertising 数据集**（`advertising.csv`）。其中字段有：

* `ID`：样本序号（可删）
* `TV`：电视广告投放金额（千元）
* `Radio`：广播广告投放金额（千元）
* `Newspaper`：报纸广告投放金额（千元）
* `Sales`：销售额（百万元）

目标：根据 TV、Radio、Newspaper 的投入，**预测销售额**。

### 4.3.2 使用线性回归预测广告投放效果（完整代码）

In [5]:
import pandas as pd
from sklearn.preprocessing import StandardScaler          # 标准化
from sklearn.model_selection import train_test_split      # 划分数据集
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.metrics import mean_squared_error            # 评估：均方误差

# 1. 加载数据集
advertising = pd.read_csv("data/advertising.csv")
advertising.drop(advertising.columns[0], axis=1, inplace=True)  # 删除 ID 列
advertising.dropna(inplace=True)                                # 删除缺失样本

print(advertising.info())
print(advertising.head())

# 2. 划分特征和目标
X = advertising.drop("Sales", axis=1)   # TV, Radio, Newspaper
y = advertising["Sales"]                # 销售额

# 划分训练集和测试集（70% 训练，30% 测试）
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0
)

# 3. 特征标准化（很重要：有助于梯度下降收敛）
preprocessor = StandardScaler()
x_train = preprocessor.fit_transform(x_train)  # 用训练集计算均值和方差，并标准化
x_test = preprocessor.transform(x_test)        # 使用相同的均值和方差标准化测试集

# 4. 使用正规方程法（LinearRegression）
normal_equation = LinearRegression()
normal_equation.fit(x_train, y_train)

print("正规方程法解得模型系数:", normal_equation.coef_)
print("正规方程法解得模型偏置:", normal_equation.intercept_)

# 5. 使用随机梯度下降法（SGDRegressor）
gradient_descent = SGDRegressor(random_state=0)
gradient_descent.fit(x_train, y_train)

print("随机梯度下降法解得模型系数:", gradient_descent.coef_)
print("随机梯度下降法解得模型偏置:", gradient_descent.intercept_)

# 6. 使用均方误差评估两种模型在测试集上的表现
y_pred_ne = normal_equation.predict(x_test)
y_pred_gd = gradient_descent.predict(x_test)

print("正规方程法均方误差:", mean_squared_error(y_test, y_pred_ne))
print("随机梯度下降法均方误差:", mean_squared_error(y_test, y_pred_gd))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         200 non-null    float64
 1   Radio      200 non-null    float64
 2   Newspaper  200 non-null    float64
 3   Sales      200 non-null    float64
dtypes: float64(4)
memory usage: 6.4 KB
None
      TV  Radio  Newspaper  Sales
0  230.1   37.8       69.2   22.1
1   44.5   39.3       45.1   10.4
2   17.2   45.9       69.3    9.3
3  151.5   41.3       58.5   18.5
4  180.8   10.8       58.4   12.9
正规方程法解得模型系数: [3.68471841 3.05065643 0.03809335]
正规方程法解得模型偏置: 14.355714285714287
随机梯度下降法解得模型系数: [3.67804191 3.03289512 0.05802398]
随机梯度下降法解得模型偏置: [14.33744876]
正规方程法均方误差: 3.691394845698609
随机梯度下降法均方误差: 3.6897573473002665


**结果说明**

* 两种方法（`LinearRegression` 和 `SGDRegressor`）本质上都是在拟合同一个线性模型，只是**求解方式不同**：

  * `LinearRegression`：解析解（正规方程）
  * `SGDRegressor`：迭代法（随机梯度下降）
* 对比两个模型的 MSE，可以直观地看出：

  * 在数据不太大时，解析解往往简单稳定
  * 在大规模数据上，SGD 更灵活、可在线更新

**应用场景总结**

通过这个案例，我们可以用线性回归解决：

> “不同广告渠道多投入一点，会让销售额多多少？”
> “给定预算，怎样调整 TV/Radio/Newspaper 的投放结构，才能带来更高的销售？”

这就是线性回归在商业场景中的典型用法。